In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.coactivity as coactivity
from lib.common.visualization import merge_image_sequence_movie

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yaro/neuronal-raw-pooled'
# params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('sych_result_activity.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
print(dataDB.mice)
print(dataDB.dataTypes)
print(dataDB.trialTypeNames)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
areaDict = {
    'anterior' : np.arange(12),
    'posterior' : np.arange(12, 24),
    'medial' : np.arange(24, 48),
}

dataDB.plot_area_clusters(fig, ax, areaDict, haveLegend=True)

# 1. Significance

## 1.1. Correlation plots


In [ ]:
argSweepDict = {
    'datatype':  ['bn_trial', 'bn_session'],
    'trialType': ['None', 'iGO', 'iNOGO', 'iMISS', 'iFA'],
    'intervName': 'auto',
    'performance': 'auto'
}

argSweepDictSubpre = {
    'trialType': ['None', 'iGO', 'iNOGO', 'iMISS', 'iFA'],
    'intervName': 'auto',
    'performance': 'auto'
}

exclQueryLst = []

In [ ]:
coactivity.plot_corr_mouse(dataDB, mc, 'corr', 'intervName', nDropPCA=None, dropChannels=[21],
                           haveBrain=True, haveMono=False, exclQueryLst=exclQueryLst, **argSweepDict)

In [ ]:
coactivity.plot_corr_mouse(dataDB, mc, 'corr', 'trialType', nDropPCA=None, dropChannels=[21],
                           haveBrain=True, haveMono=False, exclQueryLst=exclQueryLst, **argSweepDict)

In [ ]:
coactivity.plot_corr_mousephase_subpre(dataDB, mc, 'corr', nDropPCA=None, dropChannels=[21],
                                       exclQueryLst=exclQueryLst, **argSweepDictSubpre)

In [ ]:
coactivity.plot_corr_mousephase_submouse(dataDB, mc, 'corr', dropChannels=[21],
                                         exclQueryLst=exclQueryLst, **argSweepDict)

**Drop first PCA and explore result**

In [ ]:
coactivity.plot_corr_mouse(dataDB, mc, 'corr', 'intervName', nDropPCA=1, dropChannels=[21], clusterParam=0.95,
                           haveBrain=True, haveMono=False, exclQueryLst=exclQueryLst, **argSweepDict)

In [ ]:
coactivity.plot_corr_mouse(dataDB, mc, 'corr', 'trialType', nDropPCA=1, dropChannels=[21],
                           haveBrain=True, haveMono=False, exclQueryLst=exclQueryLst, **argSweepDict)

In [ ]:
coactivity.plot_corr_mousephase_subpre(dataDB, mc, 'corr', nDropPCA=1, dropChannels=[21],
                                       exclQueryLst=exclQueryLst, **argSweepDictSubpre)

In [ ]:
coactivity.plot_corr_mousephase_submouse(dataDB, mc, 'corr', nDropPCA=1, dropChannels=[21], 
                                         exclQueryLst=exclQueryLst, **argSweepDict)

## 1.4 Plot correlation movies

In [ ]:
dataKWArgs = {'trialType': 'auto', 'performance': 'auto', 'datatype': ['bn_trial', 'bn_session']}
plotKWArgs = {'limits': [-1, 1]}

for nDropPCA in [None, 1]:
    coactivity.plot_corr_movie_mousetrialtype(dataDB, mc, 'corr', dataKWArgs, plotKWArgs,
                                              exclQueryLst=exclQueryLst, haveDelay=False, nDropPCA=nDropPCA)

In [ ]:
for datatype in ['bn_trial', 'bn_session']:
    for performance in ['naive', 'expert']:
        for dropPCA in ['None', '1']:
            prefixPath = 'pics/corr/mousetrialType/dropPCA_' + str(nDropPCA) + '/movies/'
            
            plotSuffix = performance + '_' + datatype + '_'
            merge_image_sequence_movie(prefixPath + plotSuffix, '.png', 0, 200,
                                       trgPathName=None, deleteSrc=True)

## 1.2 PCA exploration

# 2. Consistency
## 2.1 Direct correlation metrics

In [ ]:
coactivity.plot_corr_consistency_l1_mouse(dataDB, intervDict, nDropPCA=None, performance='naive', dropChannels=[21])
coactivity.plot_corr_consistency_l1_mouse(dataDB, intervDict, nDropPCA=None, performance='expert', dropChannels=[21])

In [ ]:
coactivity.plot_corr_consistency_l1_mouse(dataDB, intervDict, nDropPCA=1, performance='naive', dropChannels=[21])
coactivity.plot_corr_consistency_l1_mouse(dataDB, intervDict, nDropPCA=1, performance='expert', dropChannels=[21])

In [ ]:
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='naive', dropChannels=[21], trialTypes=['iGO', 'iNOGO'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='naive', dropChannels=[21], trialTypes=['iGO', 'iNOGO'], datatype='bn_session')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='expert', dropChannels=[21], trialTypes=['iGO', 'iNOGO'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='expert', dropChannels=[21], trialTypes=['iGO', 'iNOGO'], datatype='bn_session')

In [ ]:
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='naive', dropChannels=[21], nDropPCA=1, trialTypes=['iGO', 'iNOGO'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='naive', dropChannels=[21], nDropPCA=1, trialTypes=['iGO', 'iNOGO'], datatype='bn_session')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='expert', dropChannels=[21], nDropPCA=1, trialTypes=['iGO', 'iNOGO'], datatype='bn_trial')
coactivity.plot_corr_consistency_l1_trialtype(dataDB, intervDict, performance='expert', dropChannels=[21], nDropPCA=1, trialTypes=['iGO', 'iNOGO'], datatype='bn_session')

In [ ]:
intervDictNoPre = intervDict.copy()
del intervDictNoPre['PRE']

In [ ]:
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, dropChannels=[21], performance='naive', datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, dropChannels=[21], performance='naive', datatype='bn_session')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, dropChannels=[21], performance='expert', datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, dropChannels=[21], performance='expert', datatype='bn_session')

In [ ]:
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, dropChannels=[21], performance='naive', datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, dropChannels=[21], performance='naive', datatype='bn_session')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, dropChannels=[21], performance='expert', datatype='bn_trial')
coactivity.plot_corr_consistency_l1_phase(dataDB, intervDictNoPre, nDropPCA=1, dropChannels=[21], performance='expert', datatype='bn_session')

## 2.1. PCA consistency over mice
### 2.1.1. Angle-based consistency

Tasks
  * Explained variance by phase/session/mouse/trialType
     * Do not separate phases, its meaningless. Compute PCA for all timesteps, then see proj differences for phases
     * Implement HAC correction

  * Global PCA shifts vs session

Approaches:
  * Eval PCA over all data, select strongest components, plot components as function of cofound
  * Eval PCA for cofounds, compare PCA
  
**Plots**:
* Cosine-squared matrix $C^2_{ij} = (R^{1}_{ik}R^{2}_{jk})^2$, where $R^l$ is the PCA-transform
* Consistency metric $E = e^1_i e^2_j C^2_{ij}$, where $e^l$ are the eigenvalues

**Problem**:
The consistency metric $E$ has all necessary ingredients (angles, eigenvalues), but it is not mathematically clear that it behaves the desired way. Solid theory is required for this metric to be useful.

**Alternative approach**:
Try consistency metric $H(\frac{C^2_{ij}}{N})$. Should be great at measuring the sparsity of basis coupling. The challenge is to include eigenvalue priority into this metric, since spread of weak eigenvalues is not as relevant as spread of strong ones.

In [ ]:
coactivity.plot_pca_alignment_bymouse(dataDB, datatype='bn_session', trialType=None)

### 2.1.2. Eigenvalue-based consistency

* Let $x_1$, $x_2$ be some datasets
* Let $R_1$, $R_2$ be the corresponding PCA-transforms 
* Find total variances
    - $V_1 = \sum_i eig_i(x_1) = tr(cov(x_1)) = \sum_i cov_{ii}(x_1)$
    - $V_2 = \sum_i eig_i(x_2) = tr(cov(x_2)) = \sum_i cov_{ii}(x_2)$
* Find explained variances
    - $e_1 = eval(cov(x_1)) = diag(cov(R_1 x_1))$
    - $e_2 = eval(cov(x_2)) = diag(cov(R_2 x_2))$
* Find explained variances using wrong bases
    - $e_{12} = diag(cov(R_2 x_1))$
    - $e_{21} = diag(cov(R_1 x_2))$
* Find representation errors in explained variance ratios
    - $\epsilon_1 = \frac{\sum_i |e^1_i - e^{12}_i|}{2 V_1}$
    - $\epsilon_2 = \frac{\sum_i |e^2_i - e^{21}_i|}{2 V_2}$



* TODO: iter trialType=[hit, cr, all]
* TODO: iter perf=[naive,expert,all]

In [ ]:
coactivity.plot_pca_consistency(dataDB, dropChannels=[21])

In [ ]:
coactivity.plot_pca_consistency(dataDB, dropChannels=[21], dropFirst=1)

## 2.2. PCA consistency over phases
### 2.2.1 Angle-based consistency

In [ ]:
coactivity.plot_pca_alignment_byphase(dataDB, intervNames=['TEX', 'REW'], datatype='bn_trial', trialType=None)